In [ ]:
import os
import cv2
import numpy as np
import networkx as nx
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

: 

In [2]:
base_dir = '/Users/alyssonamaral/Documents/gcn/images'

dogs_dir = os.path.join(base_dir, 'Dog')
cats_dir = os.path.join(base_dir, 'Cat')

def load_and_resize_images(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, target_size)
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)  # Converter BGR para RGB
            img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).float()  # Converter para tensor e permutar dimensões
            images.append(img_tensor)
    return images

dog_images = load_and_resize_images(dogs_dir)
cat_images = load_and_resize_images(cats_dir)

Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9
Corrupt JPEG data: 254 extraneous bytes before marker 0xd9
Corrupt JPEG data: 399 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9


In [10]:
dog_images[0].shape

torch.Size([3, 256, 256])

In [15]:
def image_to_graph(image):
    channels, height, width = image.shape
    graph = nx.Graph()

    for i in range(height):
        for j in range(width):
            rgb_values = image[:, i, j].float()  # Extrai o valor RGB de cada pixel no formato (C, H, W)
            graph.add_node((i, j), rgb=rgb_values)

    for i in range(height):
        for j in range(width):
            neighbors = [
                (i-1, j), (i+1, j), (i, j-1), (i, j+1),
                (i-1, j-1), (i-1, j+1), (i+1, j-1), (i+1, j+1)
            ]
            for ni, nj in neighbors:
                if 0 <= ni < height and 0 <= nj < width:
                    diff = torch.norm(image[:, i, j].float() - image[:, ni, nj].float())
                    graph.add_edge((i, j), (ni, nj), weight=diff)
    return graph


In [ ]:

class MessagePassingLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MessagePassingLayer, self).__init__()
        # Aqui, ajustamos para 2 * input_dim para refletir o vetor concatenado (nó + vizinhos)
        self.message_mlp = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, graph):
        new_node_features = {}
        
        for node in graph.nodes:
            node_feature = graph.nodes[node]["rgb"]
            neighbor_features = [graph.nodes[neighbor]["rgb"] for neighbor in graph.neighbors(node)]
            
            if neighbor_features:
                aggregated_message = torch.mean(torch.stack(neighbor_features), dim=0)
            else:
                aggregated_message = torch.zeros_like(node_feature)
            
            # Concatenamos a característica do nó com a mensagem agregada
            combined = torch.cat([node_feature, aggregated_message])
            new_node_feature = self.message_mlp(combined)
            new_node_features[node] = new_node_feature

        for node, new_feature in new_node_features.items():
            graph.nodes[node]["rgb"] = new_feature

        return graph


# Função para agregar os valores dos nós em um vetor
def aggregate_graph_features(graph):
    node_features = torch.stack([graph.nodes[node]["rgb"] for node in graph.nodes])
    aggregated_vector = torch.mean(node_features, dim=0)  # Agrega usando a média
    return aggregated_vector

# Classe do modelo MLP final para classificação
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

# Parâmetros para o modelo
T = 15
input_dim = 3
hidden_dim = 64
output_dim = 2

message_passing_layer = MessagePassingLayer(input_dim, hidden_dim)
classification_model = SimpleMLP(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(message_passing_layer.parameters()) + list(classification_model.parameters()), lr=0.001)

# Carrega e transforma as imagens em grafos
dog_graphs = [image_to_graph(img) for img in dog_images]
cat_graphs = [image_to_graph(img) for img in cat_images]

# Divide os grafos e rótulos em treino e teste
graphs = dog_graphs + cat_graphs
labels = [0] * len(dog_graphs) + [1] * len(cat_graphs)
train_graphs, test_graphs, train_labels, test_labels = train_test_split(graphs, labels, test_size=0.2, random_state=42)

# Função para processar os grafos e passar por rounds de message passing
def process_graphs(graphs, T):
    processed_vectors = []
    for graph in graphs:
        for _ in range(T):
            graph = message_passing_layer(graph)
        aggregated_vector = aggregate_graph_features(graph)
        processed_vectors.append(aggregated_vector)
    return torch.tensor(processed_vectors, dtype=torch.float32)

# Processa os grafos de treino e teste com a passagem de mensagem
X_train_tensor = process_graphs(train_graphs, T)
X_test_tensor = process_graphs(test_graphs, T)
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

# Configura o modelo e o treinamento
num_epochs = 20
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = classification_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Avaliação do modelo
with torch.no_grad():
    test_outputs = classification_model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f"Accuracy on test set: {accuracy * 100:.2f}%")


: 